# Fine-Tuning Flan T5 for SQL

In [19]:
import os
import torch
import time
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from transformers import TextDataset, T5ForConditionalGeneration
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, concatenate_datasets

Define configuration parameters

In [2]:
model_name='t5-small'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'    

Create model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = base_model.to(device)

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/huggingface_hub-0.24.0-py3.8.egg/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


Load datasets

In [5]:
dataset_csql_train = load_dataset("b-mc2/sql-create-context", split='train[:8%]')
dataset_csql_test = load_dataset("b-mc2/sql-create-context", split='train[-2%:-1%]')
dataset_csql_val = load_dataset("b-mc2/sql-create-context", split='train[-1%:]')

dataset_tsql_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:8%]')
dataset_tsql_train = dataset_tsql_train.remove_columns(['source', 'text'])
dataset_tsql_train = dataset_tsql_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-2%:-1%]')
dataset_tsql_test  = dataset_tsql_test.remove_columns(['source', 'text'])
dataset_tsql_test  = dataset_tsql_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-1%:]')
dataset_tsql_val   = dataset_tsql_val.remove_columns(['source', 'text'])
dataset_tsql_val   = dataset_tsql_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

In [6]:
dataset_train_merged = concatenate_datasets(
    [
        dataset_csql_train, 
        # dataset_tsql_train
        ]
    )
dataset_test_merged = concatenate_datasets(
    [
        dataset_csql_test, 
        # dataset_tsql_test
        ]
    )
dataset_val_merged = concatenate_datasets(
    [
        dataset_csql_val, 
        # dataset_tsql_val
        ]
    )

In [ ]:
dataset_train_merged.to_csv('train_merged.csv', index=False)
dataset_test_merged.to_csv('test_merged.csv', index=False)
dataset_val_merged.to_csv('val_merged.csv', index=False)

In [8]:
dataset_train_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 6286
})

In [9]:
dataset_test_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 786
})

In [10]:
dataset_val_merged

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 786
})

In [12]:
dataset_train_merged_1 = load_dataset('csv', 'merged_train.csv', split='train')
dataset_train_merged_1

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 12572
})

In [4]:
dataset = load_dataset('csv', data_files={
    "train": "train_merged.csv", 
    "test": "test_merged.csv", 
    "val": "val_merged.csv"
    })

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 6286
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 786
    })
    val: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 786
    })
})

In [7]:
dataset['test'][0]

{'context': 'CREATE TABLE table_name_41 (entrant VARCHAR, year INTEGER)',
 'question': 'Who was the entrant before 1988?',
 'answer': 'SELECT entrant FROM table_name_41 WHERE year < 1988'}

Create Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Preprocess datasets

In [7]:
def tokenize_function(sample):
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:]n"

    data_zip = zip(sample['context'], sample['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    sample['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return sample

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['question','context','answer'])


Map:   0%|          | 0/786 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 6286
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 786
    })
    val: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 786
    })
})

In [12]:
tokenized_datasets.keys()

dict_keys(['train', 'test', 'val'])

In [13]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

In [14]:
tokenized_datasets['train'][0]['input_ids'][:10]

[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]

In [15]:
tokenized_datasets['train'][0]['labels'][:10]

[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]

In [16]:
tokenized_datasets['train'].shape

(6286, 2)

In [17]:
tokenized_datasets['val'].shape

(786, 2)

In [10]:
tokenized_datasets['test'].shape

(786, 2)

Test model with zero shot prediction/inferencing

In [11]:
index = 0

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    base_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_41 (entrant VARCHAR, year INTEGER)

Question:
Who was the entrant before 1988?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT entrant FROM table_name_41 WHERE year < 1988

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Question: Who was the entrant before 1988? Answer: Who was the entrant before 1988?


Fine tune

In [13]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
    finetuned_model = finetuned_model.to('cpu')
    to_train = False

except:
    to_train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    finetuned_model = finetuned_model.to('cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/huggingface_hub-0.24.0-py3.8.egg/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


In [17]:
%%time

if to_train:
    output_dir = 'training'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,  
        no_cuda=True,                 
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['val'],
    )
    
    trainer.train()
    
    finetuned_model.save_pretrained("model")

  0%|          | 0/786 [00:00<?, ?it/s]

{'loss': 0.3147, 'learning_rate': 0.0046819338422391865, 'epoch': 0.13}
{'loss': 0.162, 'learning_rate': 0.004363867684478371, 'epoch': 0.25}
{'loss': 0.1445, 'learning_rate': 0.0040458015267175575, 'epoch': 0.38}
{'loss': 0.1364, 'learning_rate': 0.003727735368956743, 'epoch': 0.51}
{'loss': 0.1279, 'learning_rate': 0.003409669211195929, 'epoch': 0.64}
{'loss': 0.1185, 'learning_rate': 0.003091603053435115, 'epoch': 0.76}
{'loss': 0.0806, 'learning_rate': 0.0027735368956743, 'epoch': 0.89}
{'loss': 0.0572, 'learning_rate': 0.002455470737913486, 'epoch': 1.02}
{'loss': 0.0444, 'learning_rate': 0.002137404580152672, 'epoch': 1.15}
{'loss': 0.0373, 'learning_rate': 0.0018193384223918574, 'epoch': 1.27}


  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.09289006888866425, 'eval_runtime': 105.9219, 'eval_samples_per_second': 7.421, 'eval_steps_per_second': 0.472, 'epoch': 1.27}
{'loss': 0.0335, 'learning_rate': 0.0015012722646310433, 'epoch': 1.4}
{'loss': 0.0314, 'learning_rate': 0.001183206106870229, 'epoch': 1.53}
{'loss': 0.0281, 'learning_rate': 0.0008651399491094148, 'epoch': 1.65}
{'loss': 0.0274, 'learning_rate': 0.0005470737913486006, 'epoch': 1.78}
{'loss': 0.0267, 'learning_rate': 0.00022900763358778625, 'epoch': 1.91}
{'train_runtime': 23134.6719, 'train_samples_per_second': 0.543, 'train_steps_per_second': 0.034, 'train_loss': 0.08835156910292065, 'epoch': 2.0}
CPU times: user 3h 39min 40s, sys: 2h 6min 16s, total: 5h 45min 56s
Wall time: 6h 25min 34s


Load model from local folder

In [32]:
model_path = "./model"
finetuned_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [ ]:
finetuned_model

Test fine-tuned model with zero shot inferencing

In [40]:
index = 1
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_89 (engine VARCHAR, year VARCHAR)

Question:
Which engine was used in 1987?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT engine FROM table_name_89 WHERE year = 1987

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:



Test fine-tuned model with query

In [43]:
def get_sql(query):
    prompt = "translate English to SQL: %s " % query
    features = tokenizer([prompt], return_tensors='pt')
    output = finetuned_model.generate(
        input_ids=features['input_ids'],
        max_new_tokens=200
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)
    

In [44]:
query = "Which engine was used in 1987?"
get_sql(query)

'SELECT engine FROM table_1987_1987?'